In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from xgboost import XGBRegressor
from sklearn.pipeline import Pipeline

from preprocess import preprocessor, feature_selector  # your pre-built transformer

# 1. Load data
df = pd.read_csv("train.csv")
X = df.drop("SalePrice", axis=1)
y = df["SalePrice"]

# 2. Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [6]:
preprocessor.fit(X_train)

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('scaler', StandardScaler())]),
                                 ['OverallQual', 'GarageCars', 'GrLivArea',
                                  'BedroomAbvGr']),
                                ('cat',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(fill_value='NA',
                                                                strategy='constant')),
                                                 ('encoder',
                                                  OneHotEncoder(handle_unknown='ignore'))]),
                                 ['BsmtQual'])])

In [7]:
import joblib

joblib.dump(preprocessor, 'preprocessor.joblib')

['preprocessor.joblib']

In [8]:
X_train_proc = preprocessor.transform(X_train)

In [9]:
from xgboost import XGBRegressor

xgb_model = XGBRegressor(
    n_estimators = 100,
    max_depth = 3,
    learning_rate = 0.1
)

xgb_model.fit(X_train_proc, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             feature_weights=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=0.1, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=3,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=100,
             n_jobs=None, num_parallel_tree=None, ...)

In [10]:
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import numpy as np

# Example predictions
y_pred = xgb_model.predict(X_train_proc)

# R²
r2 = r2_score(y_train, y_pred)
print(f"R²: {r2:.4f}")

# MAE
mae = mean_absolute_error(y_train, y_pred)
print(f"MAE: {mae:.2f}")

# RMSE (handle manually if `squared=False` causes issues in your sklearn version)
mse = mean_squared_error(y_train, y_pred)
rmse = np.sqrt(mse)
print(f"RMSE: {rmse:.2f}")

R²: 0.8851
MAE: 18782.88
RMSE: 26175.42


In [12]:
xgb_model.save_model('xgb-model.deprecated')




/opt/conda/lib/python3.12/site-packages/xgboost/sklearn.py:1028: UserWarning: [02:23:18] WARNING: /workspace/src/c_api/c_api.cc:1398: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format is default to UBJSON in XGBoost 2.1 if not specified.
  self.get_booster().save_model(fname)


NameError: name 'xgb_model' is not defined